In [7]:

%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import redis
import json
from pprint import pprint
from joblib import dump, load

import pymongo
from datetime import datetime, timedelta, date
import pandas as pd
from multiprocessing import Pool

import numpy as np
import pymongo
import cx_Oracle
import os
import time
import plotly.express as px

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.linear_model import LogisticRegression, Lasso, RidgeClassifier
from rapidfuzz import process, fuzz
from sklearn.neural_network import MLPClassifier

from difflib import SequenceMatcher
from sklearn.cluster import KMeans

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None  # default='warn'



from sqlalchemy import event,create_engine,types
driver = 'cx_oracle'
server = '10.234.152.61' 
database = 'alfabi' 
username = 'report' 
password = 'justd0it'
engine_stmt = "oracle://%s:%s@%s/%s" % (username, password, server, database )

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from itertools import zip_longest

r = redis.Redis(host="10.25.4.67", port=6379, db=0)

    
# iterate a list in batches of size n
def batcher(iterable, n):
    args = [iter(iterable)] * n
    return zip_longest(*args)

# # in batches of 500 delete keys matching :*
# for keybatch in batcher(r.scan_iter('otp_ip_devicemodel:*'),500):
#     r.delete(*keybatch)
# # in batches of 500 delete keys matching :*
# for keybatch in batcher(r.scan_iter('otp_ip_devicemodel_prefix_phone:*'),500):
#     r.delete(*keybatch)



for i in r.scan_iter('otp_ip_deviceid:*'):
    r.delete(i)
for i in r.scan_iter('otp_ip_devicemodel:*'):
    r.delete(i)
for i in r.scan_iter('otp_phone_deviceid:*'):
    r.delete(i)
    
for i in r.scan_iter('regis_ip:*'):
    r.delete(i)
for i in r.scan_iter('regis_prefixphone:*'):
    r.delete(i)
for i in r.scan_iter('regis_name:*'):
    r.delete(i)

In [ ]:
# import redis

# r_con = redis.Redis(host="10.25.4.67", port=6379)
# r_con.get('otp_ip_deviceid:0.0.0.221_27824e6d-b056-4bc1-847d-5c65f87f89bb')

In [ ]:

r_con = redis.Redis(host='10.25.4.67',port='6379')
print(r_con.get("regis_ip:80.222.2.51"))

In [ ]:

r_con = redis.Redis(host="10.25.4.67", port=6379)
print(r_con.get("regis_prefixphone:081289454"))

In [3]:

r_con = redis.Redis(host="10.25.4.67", port=6379)
print(r_con.get("otp_ip_deviceid:20.19.1.145  - aaaaaaaa-1c3a-4159-9a19-bbbbbbbbbbbb"))

None


In [ ]:

r_con = redis.Redis(host="10.25.4.67", port=6379)
print(r_con.get("otp_ip_devicemodel:10.4.76.177 - 18aea574-1c3a-4159-9a19-f1902c3c2565"))

In [ ]:

r_con = redis.Redis(host="10.25.4.67", port=6379)
print(r_con.get("otp_phone_deviceid:081289454 - 18aea574-1c3a-4159-9a19-f1902c3c2565"))

In [ ]:
r_con.get('otp_ip_devicemodel:{}'.format('20.19.1.145 - ROG GAMING'))

In [ ]:
from register_check_prod import RegisterCheck
from pprint import pprint
m = RegisterCheck()


import time
start_time = time.time()
pprint(m.score_user_registrasi('Tony_Mcneill7828@join.com', '1-033-021-8431', '80.222.2.51', 'adawdawddaw'))
print("--- %s seconds ---" % (time.time() - start_time))

In [16]:
from register_check_prod import RegisterCheck
from pprint import pprint
m = RegisterCheck()

import time
start_time = time.time()
pprint(m.score_user_otp("20.19.1.145", "aaaaaaaa-1c3a-4159-9a19-bbbbbbbbbbbb", "099999999123", "ROG GAMING"))
print("--- %s seconds ---" % (time.time() - start_time))

pair 1 20.19.1.145 - aaaaaaaa-1c3a-4159-9a19-bbbbbbbbbbbb
pair 2 20.19.1.145 - ROG GAMING
pair 3 099999999 - aaaaaaaa-1c3a-4159-9a19-bbbbbbbbbbbb
score 1 b'13'
score 2 b'13'
score 3 b'5'
{'is_fraud': True,
 'reason': ['Unusual activity in your network'],
 'score_final': 3.8000000000000003,
 'score_parameter': {'ip_deviceid': 1.8,
                     'ip_devicemodel': 1.8,
                     'phone_deviceid': 0.2}}
--- 0.05180168151855469 seconds ---


In [ ]:
def encode(s):
    
    i = 0
    t = ''
    while i < len(s):
        if s[i].isupper():
            t += 'U'
        elif s[i].islower():
            t += 'L'
        elif s[i].isdigit():
            t += 'D'
        else:
            t += 'O'
        i += 1
        
    return t

def short_encode(s):
    i = 0
    s = encode(s)
    curr = ''
    t = ''
    
    while i < len(s):
        if curr != s[i]:
            t += s[i]
            curr = s[i]
        
        i += 1
    return t

def score_sus_email(s):

    email_input = s
    email_input = email_input.split('@')[0]
    
    pat_count_digit = encode(email_input).count('D')
    pat_encode = short_encode(email_input)
    
    if pat_encode == 'LD' and pat_count_digit >= 4:
        return 1
    elif pat_encode == 'D':
        return 1
    return 0
    
score_sus_email('123312313@gmail.com')

In [ ]:
# from gibberish_detector import detector

# path_detector_indo = '/home/server/gli-data-science/akhiyar/alfagift_alert/gibberish-detector/indo_news.model'
# Detector = detector.create_from_model(
#     path_detector_indo
# )

# def score_gibberish(input_email):
#     split_email = input_email.split('@')[0]
    
#     flag_gibberish1 = False
    
#     try:
#         if Detector.is_gibberish(split_email):
#             flag_gibberish1 = True
#     except Exception as e:
#         pass


#     if flag_gibberish1:
#         return 1
#     else:
#         return 0

# email = 'qywyeqeyqw123@gmail.com'
# score_gibberish(email)

In [ ]:
from gibberish_detector import detector

path_detector_indo = '/home/server/gli-data-science/akhiyar/alfagift_alert/gibberish-detector/indo_news.model.test'
Detector = detector.create_from_model(
    path_detector_indo
)

def score_gibberish(input_email):
    split_email = input_email.split('@')[0]
    
    score_gib = Detector.calculate_probability_of_being_gibberish(split_email)
    
    return score_gib

email = 'fadmawdwa@gmail.com'
score_gibberish(email)



In [ ]:
alfagift_member = pd.read_json('/home/server/mongo_export/alfagift_member_01jan22.json',
                              dtype='object')
alfagift_member['sapa'] = alfagift_member['sapa'].fillna('-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: x.get('store_code') \
                                                        if x != '-' else '-')
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: '-' \
                                                        if x == '' else x)

alfagift_member['created_at'] = alfagift_member['created_at'].fillna('2019-01-01T23:08:53.393Z')
alfagift_member['created_at'] = alfagift_member['created_at'].apply(lambda x: x.get('$date') \
                                                                    if type(x) != str else x)

# alfagift_member = alfagift_member[['account_card', 'sapa', 'created_at']]
alfagift_member['sapa'] = alfagift_member['sapa'].apply(lambda x: 'sapa' if x != '-' else 'not_sapa')

alfagift_member['created_at_d'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m-%d")
alfagift_member['created_at_m'] = pd.to_datetime(alfagift_member['created_at']).dt.strftime("%Y-%m")

alfagift_member['account_card'] = alfagift_member['account_card'].astype('str')

In [ ]:
# alfagift_member_sel = alfagift_member[(alfagift_member['created_at_m'] == '2022-01') |
#                                       (alfagift_member['created_at_m'] == '2022-02') |
#                                       (alfagift_member['created_at_m'] == '2022-03') |
#                                       (alfagift_member['created_at_m'] == '2022-04') ]
# alfagift_member_sel['created_at_d'] = pd.to_datetime(alfagift_member_sel['created_at']).dt.strftime("%Y-%m-%d")

# alfagift_member_sel = alfagift_member.copy()

In [ ]:
### creating label for training set


li_sus = []
for dr in pd.date_range('2021-12-01', '2022-04-20', freq='D'):
    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member[alfagift_member['created_at_d'] == sel_created]
    #print(alfagift_member_sel_d.shape)


    ## suspect by IP
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','count'))
    alfagift_member_sel_d_ip = alfagift_member_sel_d_ip[
        alfagift_member_sel_d_ip['createdFromIp_count'] >= 5].reset_index()

    sus_by_ip = pd.merge(alfagift_member_sel_d_ip, alfagift_member_sel_d, on='createdFromIp', how='left')

    ## suspect by phone
    alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
    alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
                                .agg(createdFromIp_count=('phone_sub','count'))
    alfagift_member_sel_d_phone = alfagift_member_sel_d_phone[
        alfagift_member_sel_d_phone['createdFromIp_count'] >= 5].reset_index()

    sus_by_phone = pd.merge(alfagift_member_sel_d_phone, alfagift_member_sel_d, on='phone_sub', how='left')

    
    alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
    
    
    ## saving by concating suspect aspect
    sus_all = pd.concat([
        sus_by_ip[['account_card','email','phone']], 
        sus_by_phone[['account_card','email','phone']],
        alfagift_member_sel_d[alfagift_member_sel_d['member_status'] == 27][['account_card','email']],
        alfagift_member_sel_d[alfagift_member_sel_d['score_gibberish'] > 4][['account_card','email']]
    ], axis=0)\
        .drop_duplicates()
    
    sus_all = sus_all[~sus_all['email'].str.contains('deliverysapa', case=False)]
    sus_all = sus_all[['account_card','phone','email']]
    
    
    ## inserting into table to check transaction
    df = sus_all[['account_card']]

    engine = create_engine(engine_stmt)


    dtyp = {c:types.VARCHAR(df[c].str.len().max()) for c in df.columns[df.dtypes == 'object'].tolist()}
    df.to_sql('temp_sus_member', engine, index=False, 
                                          if_exists="replace", dtype=dtyp)
    
    
    q = '''
    
        SELECT 
            rto.TRO_DATE, rto.TRO_PONTA_ID
        FROM 
            temp_sus_member tsm
            INNER JOIN RPT_TRANS_OSHOP rto
            ON rto.TRO_PONTA_ID = tsm.ACCOUNT_CARD 
    
    '''
    con_alfabi = engine.connect()
    df_exclude = pd.read_sql_query(q, con_alfabi)
    df_exclude_g =  df_exclude.groupby('tro_ponta_id').agg(tro_date_count=('tro_date','nunique'))
    df_exclude_g = df_exclude_g[df_exclude_g['tro_date_count'] >2].reset_index()

    
    con_alfabi.close()
    
    sus_all = sus_all[~sus_all['account_card'].isin(df_exclude_g['tro_ponta_id'])]
    ### end inserting into table to check transaction
    li_sus.append(sus_all)


    #break
    sus_all.to_csv(
        '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_{}'.format(sel_created)
    )

    

df_sus_all = pd.concat(li_sus).drop_duplicates()
df_sus_all.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud/sus_all_exclude_trx.csv', index=False)

In [ ]:
# alfagift_member_sel_d[alfagift_member_sel_d['score_gibberish'] > 3.8]

In [ ]:
string1 = "santi78951321santi"
string2 = ["santi12344", "aaaasanti"]


def seq_match(string1, li_string):
    # match = SequenceMatcher(None, string1, string2).find_longest_match()
    
#     sum_score = 0
    match_start = 0
    match_end = 0
    for string2 in li_string:
        match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))

#         print(match)  # -> Match(a=0, b=15, size=9)
        
        if match.size >= 3:


            if ( string1.startswith(string1[match.a:match.a + match.size]) 
                and  string2.startswith(string2[match.b:match.b + match.size]) ) :
#                 print('match in front')
                
                match_start += 1
                
            if ( string1.endswith(string1[match.a:match.a + match.size]) 
                and string2.endswith(string2[match.b:match.b + match.size]) ):
#                 print('match in end')
                
                match_end += 1
        
#         print(match_start, match_end)
#         if match_end or match_start:
#             sum_score += 1
        
        
    return match_start, match_end
    
# seq_match(string1, string2)


# ### EXAMPLE USAGE OF sequence matcher

# x = 'santi'
# alfagift_member_test = alfagift_member_sel_d[(alfagift_member_sel_d['full_name'].str.startswith(x)) | 
#                          (alfagift_member_sel_d['full_name'].str.endswith(x))][['full_name']]

# alfagift_member_test['full_name_sCount'], alfagift_member_test['full_name_eCount'] = zip(*alfagift_member_test['full_name']\
#         .apply(lambda x: seq_match(x, alfagift_member_test['full_name'])))

# alfagift_member_test.head()


def add_features(alfagift_member_sel_d):
    
    alfagift_member_sel_d['full_name_sCount'], alfagift_member_sel_d['full_name_eCount'] = \
            zip(*alfagift_member_sel_d['full_name']\
            .apply(lambda x: seq_match(x, alfagift_member_sel_d['full_name'])))
    return alfagift_member_sel_d

def parallelize_dataframe(df, func, n_cores=32):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
# ## creating feature for training set


# li_sus = []
# for dr in pd.date_range('2021-12-01', '2022-04-20', freq='D'):
#     sel_created = dr.strftime("%Y-%m-%d")

#     alfagift_member_sel_d = alfagift_member[alfagift_member['created_at_d'] == sel_created]
#     # print("{} {}".format(sel_created, alfagift_member_sel_d.shape))
#     alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
#     alfagift_member_sel_d['full_name'] = alfagift_member_sel_d['full_name'].str.lower()
    
#     ## suspect by IP
#     alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
#                             .agg(createdFromIp_count=('createdFromIp','count'))

#     alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
#                                      alfagift_member_sel_d_ip,
#                                      on='createdFromIp', how='left').fillna(0)

#     ## suspect by phone
#     alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
#                                 .agg(phoneSub_count=('phone_sub','count'))

#     alfagift_member_sel_d = pd.merge(alfagift_member_sel_d, 
#                                      alfagift_member_sel_d_phone,
#                                      on='phone_sub', how='left').fillna(0)
    
    
#     alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
#     alfagift_member_sel_d['score_sus_email'] = alfagift_member_sel_d['email'].apply(score_sus_email)
    
# #     alfagift_member_sel_d['nameMatch_count'] = alfagift_member_sel_d['full_name']\
# #     .apply(lambda x: len(process.extract(x, alfagift_member_sel_d['full_name'], 
# #             score_cutoff=70, limit=alfagift_member_sel_d.shape[0], scorer=fuzz.token_sort_ratio))-1)
 
#     alfagift_member_sel_d = parallelize_dataframe(alfagift_member_sel_d, add_features)


#     li_sus.append(alfagift_member_sel_d)
#     alfagift_member_sel_d.to_csv(
#     '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/daily/{}'.format(sel_created))

#     break

# # df_sus_all = pd.concat(li_sus)
# # df_sus_all.to_csv(
# #     '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/group_day/train_full_withname1.csv'
# #     , index=False)

In [ ]:
alfagift_member_sel_d.head()

In [ ]:
alfagift_member_sel_d.shape

In [ ]:
alfagift_member_sel_d.reset_index(drop=True).head()

In [ ]:
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','cumcount'))

In [ ]:
alfagift_member_sel_d_ip.reset_index(drop=True).head()

In [ ]:
import time
from timeit import default_timer as timer
from multiprocessing import Pool, cpu_count

def square(dr):


    sel_created = dr.strftime("%Y-%m-%d")

    alfagift_member_sel_d = alfagift_member[alfagift_member['created_at_d'] == sel_created]\
                            .iloc[:,2:].reset_index(drop=True)
    
    
    alfagift_member_sel_d['phone_sub'] =  alfagift_member_sel_d['phone'].str[0:9]
    alfagift_member_sel_d['full_name'] = alfagift_member_sel_d['full_name'].str.lower()

    ## suspect by IP
    alfagift_member_sel_d_ip = alfagift_member_sel_d.groupby(['createdFromIp'])\
                            .agg(createdFromIp_count=('createdFromIp','cumcount')).reset_index(drop=True)

    alfagift_member_sel_d = pd.concat([alfagift_member_sel_d, 
                                     alfagift_member_sel_d_ip]
                                     ,axis=1)

    ## suspect by phone
    alfagift_member_sel_d_phone = alfagift_member_sel_d.groupby(['phone_sub'])\
                                .agg(phoneSub_count=('phone_sub','cumcount')).reset_index(drop=True)

    alfagift_member_sel_d = pd.concat([alfagift_member_sel_d, 
                                     alfagift_member_sel_d_phone]
                                     ,axis=1)
    
    
    alfagift_member_sel_d['score_gibberish'] = alfagift_member_sel_d['email'].apply(score_gibberish)
    alfagift_member_sel_d['score_sus_email'] = alfagift_member_sel_d['email'].apply(score_sus_email)
    

    li_name_count = []
    for idx, row in alfagift_member_sel_d.iterrows():
        cur_name = row['full_name']

        compare_name = list(alfagift_member_sel_d.iloc[0:idx+1,:]['full_name'])


        res_compare = seq_match(cur_name, compare_name)
        li_name_count.append(res_compare)


    df_count_name = pd.DataFrame(li_name_count, columns=['full_name_sCount', 'full_name_eCount'])
    alfagift_member_sel_d = pd.concat([alfagift_member_sel_d, df_count_name], axis=1)

    alfagift_member_sel_d.to_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/daily/{}'.format(sel_created))


    return 0


start = timer()

print(f'starting computations on {cpu_count()} cores')

values = pd.date_range('2021-12-30', '2022-04-20', freq='D')

with Pool() as pool:
    res = pool.map(square, values)
    #print(res)

end = timer()
print(f'elapsed time: {end - start}')

In [ ]:
from multiprocessing import Pool, cpu_count
print(f'starting computations on {cpu_count()} cores')

In [ ]:
## creating feature using cumulative





In [ ]:
# alfagift_member_sel_d_ip1 = alfagift_member_sel_d.groupby('createdFromIp').agg(createdFromIp_count=('createdFromIp','cumcount'))

In [ ]:
# from datetime import datetime
# start_time = datetime.now()

# res = seq_match('akhiyar',list(alfagift_member_sel_d.iloc[:,:]['full_name']))
# end_time = datetime.now()
# print('Duration: {}'.format(end_time - start_time))

In [ ]:
# # possible 
# ## need to check score
# afghj56diskazani
# afghj56rdiyanto
# sfghj56upriyatno
# dfghj56wiyanti
# dfghj56wiyanti
# wfgh568ahyusaputra

In [ ]:
virgo_abuseinject = pd.read_excel(
    '/home/server/gli-data-science/ferina/Virgo_Abuse.xlsx'
    ,dtype=object,sheet_name='Inject', names=['account_card'])
# virgo_abuseuse = pd.read_excel(
#     '/home/server/gli-data-science/ferina/Virgo_Abuse.xlsx'
#     ,dtype=object,sheet_name='Use', names=['account_card'])

## load feature and label 
df_sus_all = pd.read_csv(
    '/home/server/gli-data-science/akhiyar/alfagift_alert/suspect_fraud_feature/group_day/train_full_withname1.csv'
    , dtype='object')
# df_sus_all = df_sus_all[(df_sus_all['created_at_d'] >= '2021-12-01') 
#                         & (df_sus_all['created_at_d'] <= '2022-04-17')]


sus_label = pd.read_csv('./suspect_fraud/sus_all_exclude_trx.csv', dtype='object', usecols=[0])


df_label = pd.concat([sus_label, virgo_abuseinject, virgo_abuseuse]).drop_duplicates()
df_label['label'] = 1

## merging feature with suspect label
df_train = pd.merge(df_sus_all, df_label, on='account_card', how='left')
df_train['label'] = df_train['label'].fillna(0)

df_train['score_gibberish'] = df_train['score_gibberish'].astype(float).astype(int)
df_train['score_sus_email'] = df_train['score_sus_email'].astype(int)
df_train['createdFromIp_count'] = df_train['createdFromIp_count'].astype(int)
df_train['phoneSub_count'] = df_train['phoneSub_count'].astype(int)
df_train['full_name_sCount'] = df_train['full_name_sCount'].astype(int)
df_train['full_name_eCount'] = df_train['full_name_eCount'].astype(int)

In [ ]:
wrap_col = [
'score_gibberish',
'score_sus_email',
'createdFromIp_count',
'phoneSub_count',
'full_name_sCount',
'full_name_eCount'
]
X = df_train[wrap_col]
y = df_train[['label']]

In [ ]:
#Initiating the Agglomerative Clustering model 
AC = KMeans(n_clusters=9, random_state=0)
# AC = KModes(n_clusters=5, init='Huang', n_init=5, verbose=1)

# fit model and predict clusters without PCA
yhat_AC = AC.fit_predict(X)

df_train["Clusters"]= yhat_AC


In [ ]:
df_train["Clusters"].value_counts()

In [ ]:
df_train[df_train['Clusters'] == 8].iloc[:,15:-2]\
.describe()

In [ ]:
# df_train[(df_train['Clusters'] == 8)]['created_at_d'].value_counts()

In [ ]:
df_train[(df_train['Clusters'] == 6) & (df_train['created_at_d'] > '2022-04-05')]

In [ ]:
# regressor = DecisionTreeClassifier(random_state=0, max_depth=4)
# regressor.fit(X,y)


# dump(regressor, './model/regis_dtree')
# r = export_text(regressor, feature_names=list(X), decimals=0, spacing=6)
# print("prediction rule \n\n{}".format(r))

In [ ]:
clf = LogisticRegression(random_state=0)
clf.fit(X.values, y.values.ravel())
dump(clf, './model/regis_logreg_withname1.pkl')

In [ ]:
X.values[3]

In [ ]:
li_input = [[
    2,
    0,
    5,
    0,
    100,
]]
X_test = np.array(li_input)
clf.predict_proba(X_test)

In [ ]:
clf.coef_

In [ ]:
# ## for concating feature and bobot
# pd.concat([pd.DataFrame(list(X), columns=['variabel']), 
#            pd.DataFrame(pd.Series(clf.coef_[0]), columns=['bobot'])], 1)\
# .append({'variabel':'intercept','bobot':clf.intercept_[0]},ignore_index=True)

wrap_col = [
'score_gibberish',
'score_sus_email',
'createdFromIp_count',
'phoneSub_count',
'full_name_sCount',
# 'full_name_eCount'
]
res_pred = pd.concat([
    pd.DataFrame(wrap_col, columns=['variabel']), 
    pd.DataFrame(clf.coef_[0], columns=['bobot']),
    pd.DataFrame(X_test[0], columns=['feature'])      
], 1)
# res_pred['feature'] = res_pred['feature'].astype(int) 
# res_pred['b*f'] = res_pred['bobot'] * res_pred['feature']



# dict_score_parameter = {}
# for idx, row in res_pred.iterrows():
#     dict_score_parameter[row['variabel']] = row['b*f']
res_pred

In [ ]:
# model_clf = load('./model/regis_logreg.pkl')


# X_test = pd.DataFrame()
# X_test['score_gibberish'] = [1]
# X_test['score_sus_email'] = [0]
# X_test['createdFromIp_count'] = [0]
# X_test['phoneSub_count'] = [0]

# score = model_clf.predict_proba(X_test)
# score[0][-1]


# res_pred = pd.concat([
#     pd.DataFrame(clf.feature_names_in_, columns=['variabel']), 
#     pd.DataFrame(clf.coef_[0], columns=['bobot']),
#     pd.DataFrame(X_test.values[0], columns=['feature'])      
# ], 1)
# res_pred['b*f'] = res_pred['bobot'] * res_pred['feature']

# dict_score_parameter = {}
# for idx, row in res_pred.iterrows():
#     dict_score_parameter[row['variabel']] = row['b*f']
# dict_score_parameter